In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l
import torchvision as tv
import os
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1, ratio=16):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels//4,
                               kernel_size=1, padding=0)
        self.conv2 = nn.Conv2d(num_channels//4, num_channels//4,
                               kernel_size=3, padding=1,stride=strides)
        self.conv3 = nn.Conv2d(num_channels//4, num_channels,
                               kernel_size=1, padding=0)
        
        if use_1x1conv:
            self.conv4 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv4 = None
            
        self.bn1 = nn.BatchNorm2d(num_channels//4)
        self.bn2 = nn.BatchNorm2d(num_channels//4)
        self.bn3 = nn.BatchNorm2d(num_channels)
        
        # SE layers
        self.fc1 = nn.Conv2d(num_channels, num_channels//ratio, kernel_size=1)
        self.fc2 = nn.Conv2d(num_channels//ratio, num_channels, kernel_size=1)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = F.relu(self.bn2(self.conv2(Y)))
        Y = self.bn3(self.conv3(Y))
        
        # Squeeze
        z = F.avg_pool2d(Y, Y.size(2))
        z = F.relu(self.fc1(z))
        z = F.sigmoid(self.fc2(z))

        # Excitation
        Y = Y * z 
        
        if self.conv4:
            X = self.conv4(X)
            
        Y += X
        return F.relu(Y)

In [3]:
b1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=1, padding=1))

In [4]:
def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        elif i == 0 and first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [5]:
b2 = nn.Sequential(*resnet_block(64, 256, 3, first_block=True))
b3 = nn.Sequential(*resnet_block(256, 512, 4))
b4 = nn.Sequential(*resnet_block(512, 1024, 6))
b5 = nn.Sequential(*resnet_block(1024, 2048, 3))

In [6]:
net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Dropout(0.25), nn.Linear(2048, 10))

if device == 'cuda':
	net = torch.nn.DataParallel(net)

In [9]:
X = torch.rand(size=(1, 3, 32, 32))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 32, 32])
Sequential output shape:	 torch.Size([1, 256, 32, 32])
Sequential output shape:	 torch.Size([1, 512, 16, 16])
Sequential output shape:	 torch.Size([1, 1024, 8, 8])
Sequential output shape:	 torch.Size([1, 2048, 4, 4])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 2048, 1, 1])
Flatten output shape:	 torch.Size([1, 2048])
Dropout output shape:	 torch.Size([1, 2048])
Linear output shape:	 torch.Size([1, 10])


In [10]:
def evaluate_accuracy_gpu(net, data_iter, device=None): 
    if isinstance(net, nn.Module):
        net.eval()  
        if not device:
            device = next(iter(net.parameters())).device
    metric = d2l.Accumulator(2)
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(X, list):
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y = y.to(device)
            metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [11]:
def train_net(net, train_iter, test_iter, num_epochs, lr, device):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            torch.nn.init.kaiming_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(params=net.parameters(),
                          lr=lr,
                          momentum=0.9,
                          weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(
        optimizer, milestones=[65, 130, 195], gamma=0.1)
    loss = nn.CrossEntropyLoss()
    num_batches = len(train_iter)
    for epoch in range(num_epochs):
        metric = d2l.Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
        scheduler.step()
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        print(f'------- ------- Epoch{epoch+1}/{num_epochs} ------- -------')
        print(f'loss {train_l:.5f}, train acc {train_acc:.5f}, '
          f'test acc {test_acc:.5f}')

In [12]:
shape_aug = tv.transforms.RandomResizedCrop(
    (28, 28), scale=(0.1, 1), ratio=(0.5, 2))

In [13]:
train_augs = tv.transforms.Compose([
    tv.transforms.RandomCrop(32, padding=4),
    tv.transforms.RandomHorizontalFlip(),
    tv.transforms.ToTensor(), shape_aug,
    tv.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_augs = tv.transforms.Compose([
    tv.transforms.ToTensor(), 
    tv.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [14]:
def load_cifar10(is_train, augs, batch_size):
    dataset = tv.datasets.CIFAR10(root="../data", train=is_train,
                                           transform=augs, download=True)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                    shuffle=is_train, num_workers=4)
    return dataloader

In [15]:
lr, num_epochs, batch_size = 0.01, 250, 128

In [16]:
train_iter = load_cifar10(True, train_augs, batch_size)
test_iter = load_cifar10(False, test_augs, batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
train_net(net, train_iter, test_iter, num_epochs, lr, device)

training on cuda
------- ------- Epoch1/250 ------- -------
loss 2.48973, train acc 0.23916, test acc 0.36900
------- ------- Epoch2/250 ------- -------
loss 1.83509, train acc 0.34312, test acc 0.40650
------- ------- Epoch3/250 ------- -------
loss 1.71162, train acc 0.38378, test acc 0.46400
------- ------- Epoch4/250 ------- -------
loss 1.59207, train acc 0.42652, test acc 0.48260
------- ------- Epoch5/250 ------- -------
loss 1.50219, train acc 0.45782, test acc 0.51710
------- ------- Epoch6/250 ------- -------
loss 1.43199, train acc 0.48622, test acc 0.56090
------- ------- Epoch7/250 ------- -------
loss 1.37253, train acc 0.50972, test acc 0.60940
------- ------- Epoch8/250 ------- -------
loss 1.33066, train acc 0.52294, test acc 0.58840
------- ------- Epoch9/250 ------- -------
loss 1.27732, train acc 0.54376, test acc 0.62440
------- ------- Epoch10/250 ------- -------
loss 1.23014, train acc 0.56862, test acc 0.64900
------- ------- Epoch11/250 ------- -------
loss 1.1